In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(work_dir+'/fgc_support_retri/')
from fgc_support_retri.train import *

I0309 02:51:55.391725 139693457876800 file_utils.py:39] PyTorch version 1.1.0 available.


In [3]:
train_entity_match_model(20, 12, "20200304_entity_match_lr=2e-5")

I0309 02:52:00.830930 139693457876800 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0309 02:52:01.723982 139693457876800 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0309 02:52:01.726709 139693457876800 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

KeyError: 'sp'